In [1]:
import numpy as np
import rasterio
import os
from os import listdir
from os.path import isfile, join
import glob

In [2]:
input_raster_path = r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'
outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
target_raster_path = os.path.join(outputFolder, 'FIRE_ID', 'FIREID_2000.tif')
output_raster_path = r'L:\f02_data\wildfires\spatial_data\devdata\grid1KmLAND_v2_100m_tt2.tif'
compression = 'zstd'
dtype = 'int32'

In [3]:
# apply to all rasters in a folder
infolder = os.path.join(outputFolder, 'FIRE_ID')
outfolder = os.path.join(outputFolder, 'BA_100m')
os.makedirs(outfolder, exist_ok=True)


files_ls = [f for f in glob.glob(infolder+"\*.tif")]

for i in range(len(files_ls)):
    input_file = files_ls[i]

    with rasterio.open(input_file, "r+") as src:
        raster = src.read(1)
        print(src.nodata)

        resrast = np.int32(raster)
        src.nodata = 0
        m = src.meta
        m.update({
        'dtype': 'int32',
        'compress': 'zstd'
        })

        resrast [resrast>0] = 1

        outfile_name = files_ls[i].split('\\')[-1]
        outfile_path = os.path.join(outfolder,outfile_name)

        with rasterio.open(outfile_path, 'w',  **m) as dst:
            dst.write(resrast,1)

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [5]:
import arcpy
from arcpy.sa import *
import os

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
infolder = os.path.join(outputFolder, 'BA_100m') # Burnt area where pixel values is 1 when there is a wildfire
outfolder = os.path.join(outputFolder, 'BA_1km')
os.makedirs(outfolder, exist_ok=True)

refgrid_shp = r"S:\Common workspace\ETCULS\DATA_DELIVERABLES\Reference\Masks\SDI_pending\f02_data\original delivery\Grid1km_CLC_LAND_UNION_shp\Grid1km_CLC_LAND_UNION.shp"
refgrid =  r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'

arcpy.env.snapRaster = refgrid
arcpy.env.workspace = infolder
#rasters = arcpy.ListRasters("BA*", "TIF")
rasters = arcpy.ListRasters()

# rasterize the ref grid polygon
if not arcpy.Exists(refgrid):
    arcpy.conversion.PolygonToRaster(refgrid_shp, "FID", refgrid, "MAXIMUM_AREA", "NONE", 1000, "BUILD")

for rast in rasters[1:]:
    print(rast)
    output = os.path.join(outfolder,'BA_'+rast.split('.')[0][-4:] + '_1km.tif')
    
    if not arcpy.Exists(output):
        filein = os.path.join(infolder,rast)
        inRas = arcpy.Raster(filein)
        refgrid = arcpy.Raster(refgrid)
        
        # compute number of fires in each 1km cell grid
        arcpy.env.cellSize = 100
        outZonalStats = arcpy.sa.ZonalStatistics(refgrid, "Value", inRas, "SUM", "DATA", "CURRENT_SLICE", 90, "AUTO_DETECT")
        
        ##Resample to 1km:
        arcpy.env.cellSize = 1000
        arcpy.env.workspace = outfolder
        arcpy.management.Resample(outZonalStats,'BA_'+rast.split('.')[0][-4:] + '_1km.tif', "1000 1000", "MAJORITY")



FIREID_2001.tif


ExecuteError: Failed to execute. Parameters are not valid.
ERROR 001847: Majority resample does not support floating point data type. Please choose another resample method.
Failed to execute (Resample).


In [6]:
rasters

['FIREID_2000.tif',
 'FIREID_2001.tif',
 'FIREID_2002.tif',
 'FIREID_2003.tif',
 'FIREID_2004.tif',
 'FIREID_2005.tif',
 'FIREID_2006.tif',
 'FIREID_2007.tif',
 'FIREID_2008.tif',
 'FIREID_2009.tif',
 'FIREID_2010.tif',
 'FIREID_2011.tif',
 'FIREID_2012.tif',
 'FIREID_2013.tif',
 'FIREID_2014.tif',
 'FIREID_2015.tif',
 'FIREID_2016.tif',
 'FIREID_2017.tif',
 'FIREID_2018.tif',
 'FIREID_2019.tif',
 'FIREID_2020.tif',
 'FIREID_2021.tif',
 'FIREID_2022.tif',
 'FIREID_2023.tif',
 'FIREID_2024.tif']

In [7]:
import arcpy
from arcpy.sa import *
import os

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
infolder = os.path.join(outputFolder, 'BA_100m') # Burnt area where pixel values is 1 when there is a wildfire
outfolder = os.path.join(outputFolder, 'BA_1km')
os.makedirs(outfolder, exist_ok=True)

refgrid_shp = r"S:\Common workspace\ETCULS\DATA_DELIVERABLES\Reference\Masks\SDI_pending\f02_data\original delivery\Grid1km_CLC_LAND_UNION_shp\Grid1km_CLC_LAND_UNION.shp"
refgrid =  r'L:\f02_data\wildfires\spatial_data\input\grid1KmLAND_v2.tif'

arcpy.env.snapRaster = refgrid
arcpy.env.workspace = infolder
#rasters = arcpy.ListRasters("BA*", "TIF")
rasters = arcpy.ListRasters()

# rasterize the ref grid polygon
if not arcpy.Exists(refgrid):
    arcpy.conversion.PolygonToRaster(refgrid_shp, "FID", refgrid, "MAXIMUM_AREA", "NONE", 1000, "BUILD")
for rast in rasters[1:]:
    print(rast)
    filein = os.path.join(infolder,rast)
    inRas = arcpy.Raster(filein)
    mk1 = Con(~IsNull(inRas), 1)
    aggr = arcpy.sa.Aggregate(mk1, 10, "SUM", "EXPAND", "DATA"); 
    #path_temp = os.path.join(path_gdb,rast.split('.')[0] + '_1km')
    #aggr.save(path_temp)
    arcpy.management.CopyRaster(aggr,os.path.join(outfolder,rast.split('.')[0] + '_1km.tif'),"DEFAULTS","0","0","","","8_BIT_UNSIGNED")


FIREID_2001.tif
FIREID_2002.tif
FIREID_2003.tif
FIREID_2004.tif


KeyboardInterrupt: 

In [8]:
with arcpy.EnvManager(scratchWorkspace=r"L:\f02_data\wildfires\spatial_data\devdata"):
    out_raster = arcpy.sa.Aggregate(
        in_raster="FIREID_2001.tif",
        cell_factor=10,
        aggregation_type="SUM",
        extent_handling="EXPAND",
        ignore_nodata="DATA"
    )
    out_raster.save(r"L:\f02_data\wildfires\spatial_data\devdata\BA2001_agg_tt.tif")

249

In [17]:
import arcpy
from arcpy.sa import *
import os

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
infolder = os.path.join(outputFolder, 'BA_100m') # Burnt area where pixel values is 1 when there is a wildfire
outfolder = os.path.join(outputFolder, 'BA_1km_v2')
os.makedirs(outfolder, exist_ok=True)

arcpy.env.workspace = infolder
rasters = arcpy.ListRasters()

for rast in rasters[1:2]:
    print(rast)
    filein = os.path.join(infolder,rast)
    inRas = arcpy.Raster(filein)
    arcpy.env.extent = inRas

    with arcpy.EnvManager(scratchWorkspace=r"L:\f02_data\wildfires\spatial_data\output"):
        out_raster = arcpy.sa.Aggregate(
            in_raster=inRas,
            cell_factor=10.0,
            aggregation_type="SUM",
            extent_handling="EXPAND",
            ignore_nodata="DATA"
        )
        out_raster.save(os.path.join(outfolder,'BA'+ rast.split('_')[1].split('.')[0]+ '_1km_tt4.tif'))


FIREID_2001.tif


In [ ]:
import arcpy
from arcpy.sa import *
import os

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
infolder = os.path.join(outputFolder, 'BA_100m') # Burnt area where pixel values is 1 when there is a wildfire
outfolder = os.path.join(outputFolder, 'BA_1km_v2')
os.makedirs(outfolder, exist_ok=True)

arcpy.env.workspace = infolder
rasters = arcpy.ListRasters()

for rast in rasters:
    print(rast)
    filein = os.path.join(infolder,rast)
    inRas = arcpy.Raster(filein)
    arcpy.env.extent = inRas

    with arcpy.EnvManager(scratchWorkspace=r"L:\f02_data\wildfires\spatial_data\output"):
        out_raster = arcpy.sa.Aggregate(
            in_raster=inRas,
            cell_factor=10,
            aggregation_type="SUM",
            extent_handling="EXPAND",
            ignore_nodata="DATA"
        )
        out_raster.save(os.path.join(outfolder,'BA'+ rast.split('_')[1].split('.')[0]+ '_1km.tif'))

In [ ]:
import arcpy
from arcpy.sa import *
import os

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
infolder = os.path.join(outputFolder, 'NrFires_1km') # Burnt area where pixel values is 1 when there is a wildfire
outfolder = os.path.join(outputFolder, 'NFires_1km_aggr')
os.makedirs(outfolder, exist_ok=True)

arcpy.env.workspace = infolder
rasters = arcpy.ListRasters()

for rast in rasters:
    print(rast)
    filein = os.path.join(infolder,rast)
    inRas = arcpy.Raster(filein)
    arcpy.env.extent = inRas

    with arcpy.EnvManager(scratchWorkspace=r"L:\f02_data\wildfires\spatial_data\output"):
        outraster_path = os.path.join(outfolder,'NFires'+ rast.split('_')[2].split('.')[0]+ '_1km.tif')
        if not os.path.exists(outraster_path):
            out_raster = arcpy.sa.Aggregate(
                in_raster=inRas,
                cell_factor=10,
                aggregation_type="MAXIMUM",
                extent_handling="EXPAND",
                ignore_nodata="DATA"
            )
            out_raster.save(outraster_path)

In [18]:
import arcpy
from arcpy.sa import *
import os

# Check out the Spatial Analyst extension
arcpy.CheckOutExtension("Spatial")

outputFolder = r'L:\f02_data\wildfires\spatial_data\output'
infolder = os.path.join(outputFolder, 'NrFires_1km') # Burnt area where pixel values is 1 when there is a wildfire
outfolder = os.path.join(outputFolder, 'NFires_1km_aggr')
os.makedirs(outfolder, exist_ok=True)

arcpy.env.workspace = infolder
rasters = arcpy.ListRasters()

for rast in rasters:
    print(rast)

NrFires_1km_2000.tif
NrFires_1km_2001.tif
NrFires_1km_2002.tif
NrFires_1km_2003.tif
NrFires_1km_2004.tif
NrFires_1km_2005.tif
NrFires_1km_2006.tif
NrFires_1km_2007.tif
NrFires_1km_2008.tif
NrFires_1km_2009.tif
NrFires_1km_2010.tif
NrFires_1km_2011.tif
NrFires_1km_2012.tif
NrFires_1km_2013.tif
NrFires_1km_2014.tif
NrFires_1km_2015.tif
NrFires_1km_2016.tif
NrFires_1km_2017.tif
NrFires_1km_2018.tif
NrFires_1km_2019.tif
NrFires_1km_2020.tif
NrFires_1km_2021.tif
NrFires_1km_2022.tif
NrFires_1km_2023.tif
NrFires_1km_2024.tif


In [20]:
rast.split('_')[2].split('.')[0]

'2024'